In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classesQ8 = base.iloc[:1400000, 20:28].values
classesQ8 = np.reshape(classesQ8, (2000, 700, 8))
print(classesQ8.shape)

classesQ3 = base.iloc[:1400000, 28:31].values
classesQ3 = np.reshape(classesQ3, (2000, 700, 3))
print(classesQ3.shape)

(2000, 700, 20)
(2000, 700, 8)
(2000, 700, 3)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNGRU, Bidirectional

In [5]:
def criarRede(saida):
    model = Sequential()
  
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
  
    model.add(Dense(saida, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test, saida):
    model.fit(x_train, y_train, epochs = 100, verbose = 1)
  
    predicted = model.predict(x_test)
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], saida))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], saida))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste
    
    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [7]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

accu = []
precisions = []
recalls = []
f1 = []

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede(8)
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classesQ8[train_index],
                           previsores[test_index], classesQ8[test_index], 8)

model.summary()

print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

accu = []
precisions = []
recalls = []
f1 = []

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede(3)
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classesQ3[train_index],
                           previsores[test_index], classesQ3[test_index], 3)
    
model.summary()

print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

W0823 09:24:54.509188 11592 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0823 09:24:54.514174 11592 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0823 09:24:54.516168 11592 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    2    3 ... 1996 1997 1999] TEST: [   1    7    8    9   10   17   20   25   27   29   37   43   45   48
   61   77   80   84   89   90   92   95   97   98  102  111  114  123
  125  129  137  139  142  146  153  164  165  168  176  189  193  194
  207  208  209  215  217  219  221  222  239  249  250  255  258  259
  263  269  271  275  278  279  281  282  297  298  313  335  339  340
  344  347  348  351  352  358  360  369  373  375  383  393  398  399
  404  405  406  409  410  425  426  440  444  446  470  474  477  482
  490  497  499  506  507  512  513  516  518  522  529  534  535  536
  550  553  559  569  570  573  574  579  582  596  598  600  606  611
  616  617  622  631  635  636  639  641  643  649  652  656  661  662
  669  670  672  675  687  688  690  696  711  712  723  729  740  742
  745  747  754  772  788  789  791  799  809  814  817  818  827  838
  839  840  841  853  855  862  869  870  872  875  877  884  893  895
  898  904  905  907  911  9

W0823 09:24:56.470973 11592 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 23s 15ms/sample - loss: 0.5858 - acc: 0.1597
Epoch 2/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.5354 - acc: 0.1714
Epoch 3/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.5244 - acc: 0.1767
Epoch 4/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.5197 - acc: 0.1787
Epoch 5/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.5140 - acc: 0.1815
Epoch 6/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.5110 - acc: 0.1827
Epoch 7/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.5054 - acc: 0.1850
Epoch 8/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.5018 - acc: 0.1866
Epoch 9/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.4968 - acc: 0.1885
Epoch 10/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 

1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2984 - acc: 0.2576
Epoch 81/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2965 - acc: 0.2583
Epoch 82/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2943 - acc: 0.2588
Epoch 83/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2940 - acc: 0.2588
Epoch 84/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2938 - acc: 0.2589
Epoch 85/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2915 - acc: 0.2599
Epoch 86/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2897 - acc: 0.2604
Epoch 87/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2892 - acc: 0.2605
Epoch 88/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2893 - acc: 0.2605
Epoch 89/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.28

...

1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1362 - acc: 0.9429
Epoch 94/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1346 - acc: 0.9438
Epoch 95/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1343 - acc: 0.9441
Epoch 96/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1341 - acc: 0.9442
Epoch 97/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1332 - acc: 0.9447
Epoch 98/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1318 - acc: 0.9452
Epoch 99/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1314 - acc: 0.9451
Epoch 100/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1308 - acc: 0.9456
              precision    recall  f1-score   support

           0       0.71      0.74      0.72     41747
           1       0.72      0.63      0.67     23623
           2       0.77  

1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1904 - acc: 0.9190
Epoch 48/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1874 - acc: 0.9203
Epoch 49/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1853 - acc: 0.9214
Epoch 50/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1839 - acc: 0.9219
Epoch 51/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1822 - acc: 0.9228
Epoch 52/100
1600/1600 [==============================] - 20s 13ms/sample - loss: 0.1807 - acc: 0.9234
Epoch 53/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1775 - acc: 0.9250
Epoch 54/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1757 - acc: 0.9256
Epoch 55/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1750 - acc: 0.9261
Epoch 56/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.17

1600/1600 [==============================] - 22s 14ms/sample - loss: 0.3454 - acc: 0.7165
Epoch 2/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.3151 - acc: 0.8567
Epoch 3/100
1600/1600 [==============================] - 20s 13ms/sample - loss: 0.3076 - acc: 0.8608
Epoch 4/100
1600/1600 [==============================] - 20s 13ms/sample - loss: 0.3041 - acc: 0.8626
Epoch 5/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.3054 - acc: 0.8616
Epoch 6/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.3021 - acc: 0.8632
Epoch 7/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2987 - acc: 0.8646
Epoch 8/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2976 - acc: 0.8656
Epoch 9/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2953 - acc: 0.8670
Epoch 10/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2926 - acc

1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1468 - acc: 0.9385
Epoch 81/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1449 - acc: 0.9394
Epoch 82/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1449 - acc: 0.9391
Epoch 83/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1445 - acc: 0.9397
Epoch 84/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1427 - acc: 0.9402
Epoch 85/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1427 - acc: 0.9406
Epoch 86/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1424 - acc: 0.9405
Epoch 87/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1411 - acc: 0.9412
Epoch 88/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1406 - acc: 0.9413
Epoch 89/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.13

1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2170 - acc: 0.9072
Epoch 35/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.2126 - acc: 0.9090
Epoch 36/100
1600/1600 [==============================] - 20s 13ms/sample - loss: 0.2109 - acc: 0.9097
Epoch 37/100
1600/1600 [==============================] - 20s 13ms/sample - loss: 0.2064 - acc: 0.9115
Epoch 38/100
1600/1600 [==============================] - 20s 13ms/sample - loss: 0.2048 - acc: 0.9123
Epoch 39/100
1600/1600 [==============================] - 20s 13ms/sample - loss: 0.2026 - acc: 0.9134
Epoch 40/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1987 - acc: 0.9151
Epoch 41/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1971 - acc: 0.9160
Epoch 42/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.1968 - acc: 0.9160
Epoch 43/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.19